In [8]:
import numpy as np
from ikpy.chain import Chain

# URDF 경로
URDF_PATH = "/home/jetcobot/silver_ws/src/jetcobot_movetag/urdf/mycobot_280_m5/mycobot_280m5_with_gripper_parallel.urdf"

# last_link_vector(커스텀 오프셋) - 여기에 원하는 값 넣어서 반복 실험!
last_link_vector = np.array([-0.01689683,  0.01979151, -0.22688095])

# Chain 객체 생성
chain = Chain.from_urdf_file(
    URDF_PATH,
    base_elements=["g_base"],
    last_link_vector=last_link_vector
)

# 실제 움직이는 조인트 이름과 각도(radian) 입력(예시값)
joint_names = [
    "link2_to_link1", "link3_to_link2", "link4_to_link3",
    "link5_to_link4", "link6_to_link5", "link6output_to_link6"
]
joint_angles = [0.0, -0.1, -1.0, 0.3, 0.0, -0.6]  # 라디안 단위 예시값

# Chain의 링크 이름 목록 확인
print("Chain.links 개수:", len(chain.links))
print("Chain.links 이름:", [link.name for link in chain.links])

# 각 링크별로 조인트 값 매핑 (움직이는 것만 해당 각도를, 나머지는 0.0)
joint_angle_dict = dict(zip(joint_names, joint_angles))
all_joint_angles = [
    joint_angle_dict.get(link.name, 0.0)
    for link in chain.links
]

print("입력 조인트 배열 길이:", len(all_joint_angles))
print("적용되는 조인트 각도 배열:", all_joint_angles)

# Forward Kinematics 수행
fk_matrix = chain.forward_kinematics(all_joint_angles)
tcp_pos = fk_matrix[:3, 3]
print(f"현재 TCP 위치 (FK 결과): {tcp_pos}")

# 목표 에이프릴태그 위치 입력 (본인 환경에 맞게 수정)
tag_pos = np.array([0.2, -0.05, 0.15])
print(f"목표 AprilTag 위치: {tag_pos}")

# 차이(오프셋) 계산
diff = tag_pos - tcp_pos
print(f"TCP - AprilTag 위치 차이 (오프셋 보정 방향): {diff}")

# 새로운 예상 오프셋(제안)
new_last_link_vector = last_link_vector + diff
print(f"보정 후 last_link_vector 제안값: {new_last_link_vector}")


Chain.links 개수: 11
Chain.links 이름: ['Base link', 'g_base_to_link1', 'link2_to_link1', 'link3_to_link2', 'link4_to_link3', 'link5_to_link4', 'link6_to_link5', 'link6output_to_link6', 'link6output_to_gripper_base', 'gripper_base_to_jetcocam', 'last_joint']
입력 조인트 배열 길이: 11
적용되는 조인트 각도 배열: [0.0, 0.0, 0.0, -0.1, -1.0, 0.3, 0.0, -0.6, 0.0, 0.0, 0.0]
현재 TCP 위치 (FK 결과): [ 0.07006445 -0.05355742  0.46418098]
목표 AprilTag 위치: [ 0.2  -0.05  0.15]
TCP - AprilTag 위치 차이 (오프셋 보정 방향): [ 0.12993555  0.00355742 -0.31418098]
보정 후 last_link_vector 제안값: [ 0.11303872  0.02334893 -0.54106193]


In [16]:
import numpy as np
from ikpy.chain import Chain

# URDF 파일 경로
URDF_PATH = "/home/jetcobot/silver_ws/src/jetcobot_movetag/urdf/mycobot_280_m5/mycobot_280m5_with_gripper_parallel.urdf"
last_link_vector = np.array([0.0, 0.035, 0.048])  # 현재 사용중인 오프셋

# IK Chain 생성
chain = Chain.from_urdf_file(
    URDF_PATH,
    base_elements=["g_base"],
    last_link_vector=last_link_vector
)

# IK로 이동시킬 목표 위치(x, y, z) 입력 (에이프릴태그 좌표 등)
target_pos = np.array([0.2, -0.05, 0.15])  # 원하는 목표 위치(m 단위)

# 목표 자세(회전)는 생략하거나, 단위행렬(회전없음) 사용할 수도 있습니다.
target_orientation = np.eye(3)  # 회전 없음

# Inverse Kinematics로 조인트 각도 계산
ik_result = chain.inverse_kinematics(
    target_pos,
    target_orientation=target_orientation
)

print("IK 계산 결과(전체 link 기준):", ik_result)

# 6개 주요 관절 값만 추출 (로봇별 조인트명 매칭 필요)
joint_names = [
    "link2_to_link1", "link3_to_link2", "link4_to_link3",
    "link5_to_link4", "link6_to_link5", "link6output_to_link6"
]
radian_list = []
for link in chain.links:
    if link.name in joint_names:
        idx = chain.links.index(link)
        radian_list.append(ik_result[idx])

print("IK 계산된 6개 주요 관절(rad):", radian_list)

# FK로 실제 이동된 TCP 위치 확인 (모든 link에 각도 배열 전달)
all_joint_angles = ik_result  # chain.forward_kinematics 요구 형태임
fk_matrix = chain.forward_kinematics(all_joint_angles)
tcp_pos = fk_matrix[:3, 3]
print(f"FK 실제 TCP 위치: {tcp_pos}")

# 목표 위치와의 오차 출력
print("목표 위치:", target_pos)
print("실제 TCP 위치:", tcp_pos)
print("차이(오차):", target_pos - tcp_pos)


IK 계산 결과(전체 link 기준): [ 0.          0.34900112 -0.12006466 -0.50033145 -0.52569531 -0.62615408
  0.3930771   0.2441189   0.          0.          0.        ]
IK 계산된 6개 주요 관절(rad): [-0.12006465533390176, -0.5003314525444249, -0.5256953064229652, -0.6261540805653011, 0.39307709835339855, 0.2441189009308075]
FK 실제 TCP 위치: [ 0.19999999 -0.05        0.15000001]
목표 위치: [ 0.2  -0.05  0.15]
실제 TCP 위치: [ 0.19999999 -0.05        0.15000001]
차이(오차): [ 1.09215620e-08 -3.05923227e-09 -6.32899536e-09]


# 확인용

In [17]:
import numpy as np
from ikpy.chain import Chain

URDF_PATH = "/home/jetcobot/silver_ws/src/jetcobot_movetag/urdf/mycobot_280_m5/mycobot_280m5_with_gripper_parallel.urdf"
last_link_vector = np.array([0.0, 0.035, 0.048])  # 오프셋

chain = Chain.from_urdf_file(
    URDF_PATH,
    base_elements=["g_base"],
    last_link_vector=last_link_vector
)

target_pos = np.array([0.2, -0.05, 0.15])  # 목표 위치 입력
target_orientation = np.eye(3)

# Inverse Kinematics (IK)
ik_result = chain.inverse_kinematics(
    target_pos,
    target_orientation=target_orientation
)

print("IK 계산 결과(전체 link 기준):", ik_result)


IK 계산 결과(전체 link 기준): [ 0.          0.34900112 -0.12006466 -0.50033145 -0.52569531 -0.62615408
  0.3930771   0.2441189   0.          0.          0.        ]


In [18]:
joint_names = [
    "link2_to_link1", "link3_to_link2", "link4_to_link3",
    "link5_to_link4", "link6_to_link5", "link6output_to_link6"
]
radian_list = []
for link in chain.links:
    if link.name in joint_names:
        idx = chain.links.index(link)
        radian_list.append(ik_result[idx])

print("6개 주요 조인트(rad):", radian_list)


6개 주요 조인트(rad): [-0.12006465533390176, -0.5003314525444249, -0.5256953064229652, -0.6261540805653011, 0.39307709835339855, 0.2441189009308075]


In [21]:
from pymycobot.mycobot import MyCobot
mc = MyCobot("/dev/ttyJETCOBOT", 1000000) # MyCobot 초기화
real_angles = mc.get_radians()  # 실제 하드웨어에서 읽음
print("로봇 측정 조인트(rad):", real_angles)

# 링크 이름 기준으로 매핑 (chain.links 순서와 실제 조인트 이름이 맞는지 확인)
real_joint_dict = dict(zip(joint_names, real_angles))
all_joint_angles = [
    real_joint_dict.get(link.name, 0.0)
    for link in chain.links
]

fk_matrix = chain.forward_kinematics(all_joint_angles)
tcp_pos = fk_matrix[:3, 3]
print(f"실제 FK TCP 위치: {tcp_pos}")


Note: This class is no longer maintained since v3.6.0, please refer to the project documentation: https://github.com/elephantrobotics/pymycobot/blob/main/README.md


로봇 측정 조인트(rad): [0.101, 0.88, -1.494, -0.471, 0.012, -0.775]
실제 FK TCP 위치: [ 0.11153859 -0.05223847  0.20749738]


In [22]:
print("목표 위치:", target_pos)
print("실제 TCP 위치:", tcp_pos)
print("차이(오차):", target_pos - tcp_pos)


목표 위치: [ 0.2  -0.05  0.15]
실제 TCP 위치: [ 0.11153859 -0.05223847  0.20749738]
차이(오차): [ 0.08846141  0.00223847 -0.05749738]


# 실제 로봇 조인트와 FK 위치, 오차 확인


In [23]:
import numpy as np
from ikpy.chain import Chain
from pymycobot.mycobot import MyCobot
import math
import time

# 1. 체인 및 URDF 설정
URDF_PATH = "/home/jetcobot/silver_ws/src/jetcobot_movetag/urdf/mycobot_280_m5/mycobot_280m5_with_gripper_parallel.urdf"
last_link_vector = np.array([0.0, 0.035, 0.048])
chain = Chain.from_urdf_file(URDF_PATH, base_elements=["g_base"], last_link_vector=last_link_vector)

joint_names = [
    "link2_to_link1", "link3_to_link2", "link4_to_link3",
    "link5_to_link4", "link6_to_link5", "link6output_to_link6"
]

# 2. 목표 위치 입력
target_pos = np.array([0.2, -0.05, 0.15])
target_orientation = np.eye(3)

# 3. IK로 조인트 각도 계산
ik_result = chain.inverse_kinematics(target_pos, target_orientation=target_orientation)

# 6개 메인 조인트 값만 추출
radian_list = []
for link in chain.links:
    if link.name in joint_names:
        idx = chain.links.index(link)
        radian_list.append(ik_result[idx])

# 4. 실제 로봇 제어기 초기화 및 이동 명령
mc = MyCobot("/dev/ttyJETCOBOT", 1000000)
degree_list = [math.degrees(rad) for rad in radian_list]
mc.send_angles(degree_list, 50, _async=True)
time.sleep(3)  # 이동 대기

# 5. 이동 후 get_radians()로 실제 로봇 각도 읽기
real_angles = mc.get_radians()
print("실제로 움직인 후 측정된 조인트(rad):", real_angles)

# 6. 실측 조인트 각도와 체인 링크 순서 매핑
real_joint_dict = dict(zip(joint_names, real_angles))
all_joint_angles = [
    real_joint_dict.get(link.name, 0.0)
    for link in chain.links
]

# 7. FK로 현재 TCP 위치 계산
fk_matrix = chain.forward_kinematics(all_joint_angles)
tcp_pos = fk_matrix[:3, 3]
print("실제 조인트값 기준 FK로 계산된 TCP 위치:", tcp_pos)

# 8. 목표 위치와 오차 출력
print("목표 위치:", target_pos)
print("실제 TCP 위치:", tcp_pos)
print("차이(오차, m):", target_pos - tcp_pos)


Note: This class is no longer maintained since v3.6.0, please refer to the project documentation: https://github.com/elephantrobotics/pymycobot/blob/main/README.md
실제로 움직인 후 측정된 조인트(rad): [-0.107, -0.515, -0.548, -0.644, 0.373, 0.23]
실제 조인트값 기준 FK로 계산된 TCP 위치: [ 0.19652779 -0.04932797  0.14179198]
목표 위치: [ 0.2  -0.05  0.15]
실제 TCP 위치: [ 0.19652779 -0.04932797  0.14179198]
차이(오차, m): [ 0.00347221 -0.00067203  0.00820802]


In [25]:
import numpy as np

# (1) 실로봇 값 (mm 단위 → m 단위 변환 필요시 / 1000)
robot_tcp_mm = np.array([198.5, -68.2, 219.1])  # rx, ry, rz 제외

# (2) FK 계산 위치 (m 단위 → mm 단위로 변환)
fk_tcp_m = np.array([0.19652779, -0.04932797, 0.14179198])
fk_tcp_mm = fk_tcp_m * 1000

# (3) 목표 위치 (m 단위 → mm 단위로 변환)
target_pos_m = np.array([0.2, -0.05, 0.15])
target_pos_mm = target_pos_m * 1000

# (4) 오차 직접 계산
fk_diff_mm = target_pos_mm - fk_tcp_mm
real_diff_mm = target_pos_mm - robot_tcp_mm

print("실로봇 get_coords() 위치 (mm):", robot_tcp_mm)
print("FK 계산 위치 (mm):            ", fk_tcp_mm)
print("목표 위치 (mm):              ", target_pos_mm)
print("FK 기반 오차 (mm):           ", fk_diff_mm)
print("실로봇 실제 오차 (mm):        ", real_diff_mm)


실로봇 get_coords() 위치 (mm): [198.5 -68.2 219.1]
FK 계산 위치 (mm):             [196.52779 -49.32797 141.79198]
목표 위치 (mm):               [200. -50. 150.]
FK 기반 오차 (mm):            [ 3.47221 -0.67203  8.20802]
실로봇 실제 오차 (mm):         [  1.5  18.2 -69.1]


In [ ]:
from pymycobot.mycobot import MyCobot

mc = MyCobot("/dev/ttyJETCOBOT", 1000000)
coords = mc.get_coords()
print("로봇 엔드이펙터(TCP) 실제위치:", coords)


Note: This class is no longer maintained since v3.6.0, please refer to the project documentation: https://github.com/elephantrobotics/pymycobot/blob/main/README.md
로봇 엔드이펙터(TCP) 실베위치: [198.5, -68.2, 219.1, 166.86, 18.76, -112.91]


# 최종 테스트 코드


In [ ]:
import numpy as np
from ikpy.chain import Chain
from pymycobot.mycobot import MyCobot
import math
import time

# 1. URDF 및 IK Chain 설정
URDF_PATH = "/home/jetcobot/silver_ws/src/jetcobot_movetag/urdf/mycobot_280_m5/mycobot_280m5_with_gripper_parallel.urdf"
# last_link_vector = np.array([0.0, 0.035, 0.0155])
last_link_vector = np.array([0.0462, 0.0252, 0.0129])
chain = Chain.from_urdf_file(URDF_PATH, base_elements=["g_base"], last_link_vector=last_link_vector)

joint_names = [
    "link2_to_link1", "link3_to_link2", "link4_to_link3",
    "link5_to_link4", "link6_to_link5", "link6output_to_link6"
]

# 2. 목표 위치 및 자세 입력
# target_pos = np.array([0.2, -0.05, 0.15])  # m 단위
target_pos = np.array([0.048, -0.0643, 0.4107])  # m 단위
target_orientation = np.eye(3)

# 3. IK로 조인트 각도 계산
ik_result = chain.inverse_kinematics(target_pos, target_orientation=target_orientation)

# 4. 6개 주요 조인트 값 추출
radian_list = []
for link in chain.links:
    if link.name in joint_names:
        idx = chain.links.index(link)
        radian_list.append(ik_result[idx])

# 5. 실제 로봇 이동
mc = MyCobot("/dev/ttyJETCOBOT", 1000000)
degree_list = [math.degrees(rad) for rad in radian_list]
mc.send_angles(degree_list, 5, _async=True)
time.sleep(3)  # 로봇 이동 대기

# 6. 이동 후 실제 로봇 조인트 읽기 및 FK 위치 계산
real_angles = mc.get_radians()
real_joint_dict = dict(zip(joint_names, real_angles))
all_joint_angles = [real_joint_dict.get(link.name, 0.0) for link in chain.links]
fk_matrix = chain.forward_kinematics(all_joint_angles)
tcp_pos_fk = fk_matrix[:3, 3]

# 7. 실제 로봇의 엔드이펙터 위치 읽기
coords = mc.get_coords()  # [x, y, z, rx, ry, rz] (mm, deg)
robot_tcp_mm = np.array(coords[:3])


/home/jetcobot/.local/lib/python3.12/site-packages/ikpy/urdf/URDF.py:261: UserWarning: Joint g_base_to_link1 is of type: fixed, but has an 'axis' attribute defined. This is not in the URDF spec and thus this axis is ignored
  warnings.warn("Joint {} is of type: fixed, but has an 'axis' attribute defined. This is not in the URDF spec and thus this axis is ignored".format(joint.attrib["name"]))
/home/jetcobot/.local/lib/python3.12/site-packages/ikpy/chain.py:60: UserWarning: Link Base link (index: 0) is of type 'fixed' but set as active in the active_links_mask. In practice, this fixed link doesn't provide any transformation so is as it were inactive
  warnings.warn("Link {} (index: {}) is of type 'fixed' but set as active in the active_links_mask. In practice, this fixed link doesn't provide any transformation so is as it were inactive".format(link.name, link_index))
/home/jetcobot/.local/lib/python3.12/site-packages/ikpy/chain.py:60: UserWarning: Link g_base_to_link1 (index: 1) is of t

Note: This class is no longer maintained since v3.6.0, please refer to the project documentation: https://github.com/elephantrobotics/pymycobot/blob/main/README.md


In [6]:
# 10. 결과 출력
print("--- 통합 실험 결과 (모든 좌표 mm 단위) ---")
print(f"목표 위치:                      {target_pos}")
print(f"FK 계산 위치:                   {tcp_pos_fk}")
print(f"실로봇 엔드이펙터 실제 위치:     {robot_tcp_mm}")
print(f"FK 기반 오차(목표-FK):           {fk_diff_mm}")
print(f"실로봇 실제 오차(목표-실제):      {real_diff_mm}")

# 표로 보기 쉽도록 정리
import pandas as pd
df = pd.DataFrame({
    "목표 위치 (mm)": target_pos_mm,
    "FK 계산 위치 (mm)": tcp_pos_fk_mm,
    "실로봇 위치 (mm)": robot_tcp_mm,
    "FK 오차 (mm)": fk_diff_mm,
    "실로봇 오차 (mm)": real_diff_mm
}, index=["x", "y", "z"])
print(df)


--- 통합 실험 결과 (모든 좌표 mm 단위) ---
목표 위치:                      [ 0.048  -0.0643  0.4107]
FK 계산 위치:                   [ 0.24067245 -0.14481382  0.2495779 ]
실로봇 엔드이펙터 실제 위치:     [-31.  -41.8 408.9]
FK 기반 오차(목표-FK):           [-192.67245214   80.5138211   161.12210077]
실로봇 실제 오차(목표-실제):      [-132.7   69.8  108.7]
   목표 위치 (mm)  FK 계산 위치 (mm)  실로봇 위치 (mm)  FK 오차 (mm)  실로봇 오차 (mm)
x        48.0     240.672452        -31.0 -192.672452       -132.7
y       -64.3    -144.813821        -41.8   80.513821         69.8
z       410.7     249.577899        408.9  161.122101        108.7


In [48]:
import numpy as np
from ikpy.chain import Chain
import math
import time
from pymycobot.mycobot import MyCobot

URDF_PATH = "/home/jetcobot/silver_ws/src/jetcobot_movetag/urdf/mycobot_280_m5/mycobot_280m5_with_gripper_parallel.urdf"
last_link_vector = np.array([0.0, 0.035, 0.0248])  # 예시 오프셋, 실제 실험에 맞게
# [0.1171, 0.0155, 0.0044]
# last_link_vector = np.array([0.0, 0.035, 0.0155])
# last_link_vector = np.array([0.0462, 0.0252, 0.0129])
chain = Chain.from_urdf_file(URDF_PATH, base_elements=["g_base"], last_link_vector=last_link_vector)

joint_names = [
    "link2_to_link1", "link3_to_link2", "link4_to_link3",
    "link5_to_link4", "link6_to_link5", "link6output_to_link6"
]

target_pos = np.array([0.048, -0.0643, 0.4107])   # m 단위
# target_pos = np.array([0.2, -0.05, 0.15])  # m 단위

target_orientation = np.eye(3)
ik_result = chain.inverse_kinematics(target_pos, target_orientation=target_orientation) # IK로 각도 계산

# 1. 올바른 all_joint_angles 생성 (chain.links와 정확하게 mapping)
joint_map = {ln: ik_result[i] for i, ln in enumerate([link.name for link in chain.links])}
real_joint_dict = {name: ik_result[[link.name for link in chain.links].index(name)] for name in joint_names}
all_joint_angles = [joint_map[link.name] for link in chain.links]

# 2. radian_list = 주요 6개 조인트
radian_list = [real_joint_dict[name] for name in joint_names]

# 3. FK 예상값(이동 전, IK 결과)
fk_matrix = chain.forward_kinematics(all_joint_angles)
tcp_pos_fk = fk_matrix[:3, 3]

print("FK 예상 TCP 위치(이동 전):", tcp_pos_fk * 1000)  # mm 단위로 출력

# 4. 실제 로봇 이동
mc = MyCobot("/dev/ttyJETCOBOT", 1000000)
degree_list = [math.degrees(rad) for rad in radian_list]
mc.send_angles(degree_list, 5, _async=True)
time.sleep(3)

# 5. 로봇 도착 후 실제 조인트·실물 위치 측정
real_angles = mc.get_radians()
coords = mc.get_coords()
robot_tcp_mm = np.array(coords[:3])

/home/jetcobot/.local/lib/python3.12/site-packages/ikpy/urdf/URDF.py:261: UserWarning: Joint g_base_to_link1 is of type: fixed, but has an 'axis' attribute defined. This is not in the URDF spec and thus this axis is ignored
  warnings.warn("Joint {} is of type: fixed, but has an 'axis' attribute defined. This is not in the URDF spec and thus this axis is ignored".format(joint.attrib["name"]))
/home/jetcobot/.local/lib/python3.12/site-packages/ikpy/chain.py:60: UserWarning: Link Base link (index: 0) is of type 'fixed' but set as active in the active_links_mask. In practice, this fixed link doesn't provide any transformation so is as it were inactive
  warnings.warn("Link {} (index: {}) is of type 'fixed' but set as active in the active_links_mask. In practice, this fixed link doesn't provide any transformation so is as it were inactive".format(link.name, link_index))
/home/jetcobot/.local/lib/python3.12/site-packages/ikpy/chain.py:60: UserWarning: Link g_base_to_link1 (index: 1) is of t

FK 예상 TCP 위치(이동 전): [ 48.  -64.3 410.7]
Note: This class is no longer maintained since v3.6.0, please refer to the project documentation: https://github.com/elephantrobotics/pymycobot/blob/main/README.md


In [49]:
import numpy as np
import pandas as pd

# 1. 단위 변환
tcp_pos_fk_mm = tcp_pos_fk * 1000               # FK 예측 위치 (mm)
target_pos_mm = target_pos * 1000               # 목표 위치 (mm)
robot_tcp_mm = np.array(coords[:3])             # 실로봇 위치 (mm)

# 2. 오차 계산
fk_diff_mm = target_pos_mm - tcp_pos_fk_mm      # FK 오차 (목표 - FK 모델)
real_diff_mm = target_pos_mm - robot_tcp_mm     # 실측 오차 (목표 - 실로봇)

# 3. 출력 및 표
separator = "-" * 50
print(separator)
print("           [로봇 위치 정밀도 실험 결과]")
print(separator)
print(f"목표 위치 (mm):           {target_pos_mm.round(3)}")
print(f"FK 예측 위치 (mm):        {tcp_pos_fk_mm.round(3)}")
print(f"실로봇 위치 (mm):         {robot_tcp_mm.round(3)}")
print(separator)
print(f"FK 오차 (목표-FK, mm):    {fk_diff_mm.round(3)}")
print(f"실측 오차 (목표-실제, mm): {real_diff_mm.round(3)}")
print(separator)

# 4. 데이터프레임 표
df = pd.DataFrame({
    "Target (mm)": target_pos_mm.round(3),
    "FK Pred (mm)": tcp_pos_fk_mm.round(3),
    "Robot (mm)": robot_tcp_mm.round(3),
    "FK Err (mm)": fk_diff_mm.round(3),
    "Real Err (mm)": real_diff_mm.round(3)
}, index=["x", "y", "z"])
print("\n[축별 구체값 비교]\n")
print(df)


--------------------------------------------------
           [로봇 위치 정밀도 실험 결과]
--------------------------------------------------
목표 위치 (mm):           [ 48.  -64.3 410.7]
FK 예측 위치 (mm):        [ 48.  -64.3 410.7]
실로봇 위치 (mm):         [-24.2 -45.7 413.2]
--------------------------------------------------
FK 오차 (목표-FK, mm):    [-0. -0.  0.]
실측 오차 (목표-실제, mm): [ 72.2 -18.6  -2.5]
--------------------------------------------------

[축별 구체값 비교]

   Target (mm)  FK Pred (mm)  Robot (mm)  FK Err (mm)  Real Err (mm)
x         48.0          48.0       -24.2         -0.0           72.2
y        -64.3         -64.3       -45.7         -0.0          -18.6
z        410.7         410.7       413.2          0.0           -2.5


In [32]:
mc.get_radians()

[-0.106, -0.512, -0.537, -0.644, 0.373, 0.23]